In [7]:
from qm1.grid import *
from qm1.qmsystem import *  
# set up a grid (with vanishing boundary conditions)
grid = UniformGrid(boundary_condition="vanishing", xmin=-20., xmax=20., num=250)
# define potential to use
potential = BarrierPot(xstart=-1., xstop=+2., vstep=-1.)
# set the quantum mechanical system
qsys = QMSystem(potential, grid)

In [8]:
from qm1.operators import *
# get predefined operators that act on the wave function
op_identity = IdentityOp(qsys.grid)
op_position = PositionOp(qsys.grid)
op_momentum = MomentumOp(qsys.grid)
op_hamilton = HamiltonOp(qsys)
op_potential= PotentialOp(qsys)
# make the operators more efficient
make_efficient([op_identity, op_position, op_momentum, op_potential, op_hamilton])

In [9]:
from qm1.eigensystem import Eigensystem
import pandas

# compute some eigenstates of the Hamiltonian
eigsys = Eigensystem(qsys=qsys, operator=op_hamilton)
eigsys.show('eigensystem.png')
# survey their observables
obs = eigsys.get_obs([op_identity, op_position, op_momentum, op_hamilton])
norms, positionExps, positionVars, momenta, energyExps, energyVars = [], [], [], [], [], []
for _i in range(eigsys.num):
  norms.append(np.real(eigsys.eigstates[_i].expectation_value(op_identity)))
  positionExps.append(np.real(eigsys.eigstates[_i].expectation_value(op_position)))
  positionVars.append(np.real(eigsys.eigstates[_i].variance(op_position)))
  momenta.append(np.real(eigsys.eigstates[_i].expectation_value(op_momentum)))
  energyExps.append(np.real(eigsys.eigstates[_i].expectation_value(op_hamilton)))
  energyVars.append(np.real(eigsys.eigstates[_i].variance(op_hamilton)))

pandas.DataFrame([norms, positionExps, positionVars, momenta, energyExps, energyVars], ['norms','positions', 'positionVars', 'momenta', 'energies', 'energyVars'], range(num_states))

,0,1,2,3,4,5,6,7,8,9
norms,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
positions,5.600000e-01,5.599999e-01,-6.134480e+00,6.644389e+00,-6.080011e+00,6.482362e+00,-5.930486e+00,6.207671e+00,-5.619236e+00,5.740774e+00
positionVars,8.156850e-01,3.609795e+00,8.206813e+01,9.265017e+01,9.584922e+01,1.028851e+02,9.923732e+01,1.054978e+02,1.026486e+02,1.093322e+02
momenta,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
energies,-7.592676e-01,-1.575323e-01,1.358575e-02,1.665590e-02,5.415153e-02,6.551340e-02,1.211830e-01,1.445065e-01,2.139913e-01,2.519277e-01
energyVars,7.415034e-19,1.581451e-14,1.193310e-15,6.310218e-14,9.577347e-16,7.856482e-15,2.106194e-14,3.394065e-14,1.790632e-18,4.874292e-14
